In [1]:
import praw
from data import *
import time
import pandas as pd
import matplotlib.pyplot as plt
import squarify
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
import emoji    # removes emojis
import re   # removes links
import en_core_web_sm
import string
from datetime import datetime
import json
import numpy as np
import time

In [2]:
import nltk
nltk.downloader.download('vader_lexicon')
nltk.downloader.download('wordnet')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Venetis\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Venetis\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
reddit = praw.Reddit(user_agent="###",
                         client_id="###",
                         client_secret="###",
                         username="###",
                         password="###")

In [5]:
#####get daily and weekly trends
def data_extractor_daily_and_week(reddit,no_posts):
    '''extracts daily and weekly mentioned tickers
    Parameter: reddt: reddit obj
               no_posts: number of posts that will be searched #200 and 500 are ok
    Return:   check the return section for more information
    
    '''


    ##################################
    #Initialization
    subs = ['wallstreetbets',"stocks","stock_picks","spacs","pennystocks","finance","financialindependence","options","investing","forex","stockmarket","Shortsqueeze" ]     # sub-reddit to search

    
    # post will be considered if it satisfies the following requirements
    #upvotes = [0,0.3,0.5,0.7]
    #ups = [0,20,200,500]
    upvotes = [0]
    ups = [0]
    
    ################################### 
    #Return Items
    from collections import defaultdict
    
    #weekly
    titles = {} #a dict that contains all the titles for the given subreddit and filtering combination
    #titles = {"wsb":{(0,0):[],(0,20):[]}} #key1 subreddit #key2 filtering combination, value a list that contains the titles
    #tiles[sub]=defaultdict(list)
    
    tickers = defaultdict(lambda: defaultdict(int)) #remove two # to use it
    # a dict that contains all the ticks, for the given post filtering combinations, the value is the no appearances
    #ticks = {(0,0):{"TSLA":#count, "APPL":#count }, (0,20):{"TSLA":#count, "APPL":#count }}
    #key1 filtering combination #key2 stock , value is the #count of the tick
    
    tick_comments = defaultdict(lambda: defaultdict(list))
    #similar structure as the tickers but the value is the actual comments that correspond to that tick
    tick_dates = defaultdict(lambda: defaultdict(list))
    #similar strucutre as the tickers but the value is the date the comment was made
    
    
    #daily
    titles_d = {} #a dict that contains all the titles for the given subreddit and filtering combination
    #titles = {"wsb":{(0,0):[],(0,20):[]}} #key1 subreddit #key2 filtering combination, value a list that contains the titles
    #tiles[sub]=defaultdict(list)
    
    tickers_d = defaultdict(lambda: defaultdict(int)) #remove two # to use it
    # a dict that contains all the ticks, for the given post filtering combinations, the value is the no appearances
    #ticks = {(0,0):{"TSLA":#count, "APPL":#count }, (0,20):{"TSLA":#count, "APPL":#count }}
    #key1 filtering combination #key2 stock , value is the #count of the tick
    
    tick_comments_d = defaultdict(lambda: defaultdict(list))
    #similar structure as the tickers but the value is the actual comments that correspond to that tick
    tick_dates_d = defaultdict(lambda: defaultdict(list))
    #similar strucutre as the tickers but the value is the date the comment was made
    
    
    
    
    ###########################################################################
    for sub in subs:
        start = time.time() #time it took for on subreddit comment extraction
        print(sub)
        
        #### subreddit selection
        subreddit = reddit.subreddit(sub) #select the subreddit
        #titles[sub]=defaultdict(list)
        
        
        ###### itterate through the posts

        hot_posts = subreddit.hot( limit=no_posts)
        for post in hot_posts:
            try:
                flair = post.link_flair_text #flair tags, they explain
                author = post.author.name   #post author
                post_text = post.selftext   #actual post (text)
                
            except:
                pass
            #post info has issue skip
            #i can use continue to bypass the for itteration
              
            
            
            post.comment_sort = 'hot'   #keep the most "hot" comments
            comments = post.comments #comments of that post
            
            #get more comments
            try:
                post.comments.replace_more(limit=5)   #might not be possible to replace that many
            except:
                try:
                    post.comments.replace_more(limit=3)   #might not be possible to replace that many
                except:
                    try:
                        post.comments.replace_more(limit=1)   #might not be possible to replace that many
                    except:
                        pass
            
             
            
            #itterate through various filtering methods
            #upvotes = [0,0.3,0.50.7]
            #ups = [0,20,200,500]
                
            for upv in upvotes:
                for up in ups:
                    #keep posts that satisfy the min requirements bellow
                    if post.upvote_ratio >= upv and post.ups > up: 
                                                                              
                        #comment extraction 
                        
                        for comment in comments:
                            
                            try: #deleted comment, skip
                                auth = comment.author.name #if deleted this would give an error
                            except:
                                continue #skip comment if it is deleted
                                
                            time_creation = (time.time()- comment.created_utc)/60/60/24
                            
                            #weekly and daily after middle of function
                            if float(time_creation)<=7 : 
                                split = comment.body.split(" ") #split comment to words
                                for word in split:
                                    word = word.replace("$", "") 

                                    #stocks tickers are capitals and less or equal than 5 letters
                                    #we also check if the tick is inside the stock lists "us"
                                    if word.isupper() and len(word) <= 5 and word not in blacklist and word in us:

                                        tickers[(upv,up)][word] = tickers[(upv,up)][word] + 1 #no appearances for the given tick

                                        #further filter by date #could be done in the dataframe?
                                        tick_comments[(upv,up)][word].append(comment.body) 
                                        #comment related to that tick
                                        tick_dates[(upv,up)][word].append(datetime.fromtimestamp(comment.created_utc))
                                        #date the comment was posted
                                        
                                        #daily  
                                        if float(time_creation)<=1:
                                            
                                            tickers_d[(upv,up)][word] = tickers_d[(upv,up)][word] + 1 #no appearances for the given tick

                                            #further filter by date #could be done in the dataframe?
                                            tick_comments_d[(upv,up)][word].append(comment.body) 
                                            #comment related to that tick
                                            tick_dates_d[(upv,up)][word].append(datetime.fromtimestamp(comment.created_utc))
                                            #date the comment was posted
                                        
                            
                                        
                                        

        end = time.time()
        print("time",(end - start)/60)
        
        
                
    return tick_comments, tick_dates, tickers, tickers_d, tick_comments_d, tick_dates_d            

In [6]:
tick_comments, tick_dates, tickers, tickers_d, tick_comments_d, tick_dates_d  = data_extractor_daily_and_week(reddit,500)

wallstreetbets
time 9.780030655860902
stocks
time 8.410535824298858
stock_picks
time 3.7956371903419495
spacs
time 11.347148803869883
pennystocks
time 6.998927319049836
finance
time 5.818339415391287
financialindependence
time 14.45028451681137
options
time 9.454303336143493
investing
time 4.300775929292043
forex
time 8.71840387582779
stockmarket
time 6.313835084438324
Shortsqueeze
time 8.511406175295512


In [33]:
def find_top_n_stocks(stock_count,n,min_no_com):
    '''
    Input: 
    stock_count: dictionary that has the name of the stock as key and the number of appearances for that stock is the key
    n: number of stock with the highest count that will be selected
    min_no_com: minimum number of comments/mentions needed in order to include that stock #20 for daily #100 weekly 
    
    Output 
    ret: a list that contains the n most mentioned stocks
    '''
    sorted_dict = dict(sorted(stock_count.items(), key=lambda item: item[1], reverse=True))
    ret = []
    for stck in sorted_dict.keys():
        if sorted_dict[stck]>=min_no_com:
            ret.append(stck)
    return ret[:n]


In [34]:
daily_ticks = find_top_n_stocks(tickers_d[(0,0)],15,20)
print(f"Daily ticks: {daily_ticks}")
weekly_ticks = find_top_n_stocks(tickers[(0,0)],15,50)
print(f"Weekly ticks: {weekly_ticks}")

Daily ticks: ['CRTX', 'AAPL', 'GME', 'AMD', 'TSLA', 'BABA', 'NUZE', 'PLTR', 'SI', 'PROG', 'DKNG', 'WISH', 'MSFT', 'FB', 'NVDA']
Weekly ticks: ['AAPL', 'AMD', 'GME', 'NVDA', 'TSLA', 'BABA', 'MRNA', 'WISH', 'MSFT', 'CRTX', 'PLTR', 'DIS', 'F', 'PYPL', 'PE']


In [28]:
def df_top_trenders(daily_ticks,weekly_ticks,save):
    #save: represents whether u want the df to be saved as a csv too (True or False)
    #returns a dataframe that contains the daily and weekly trenders
    d = {"daily_trenders":daily_ticks,"weekly_trenders":weekly_ticks}
    df = pd.DataFrame(d)
    if save==True:
        df.to_csv("daily_weekly_trending_stocks.csv", index=False)        
    return df
        
    
    

In [29]:
df_top_trenders(daily_ticks,weekly_ticks,True)

,daily_trenders,weekly_trenders
0,CRTX,AAPL
1,AAPL,AMD
2,GME,GME
3,AMD,NVDA
4,TSLA,TSLA
5,BABA,BABA
6,NUZE,MRNA
7,PLTR,WISH
8,SI,MSFT
9,PROG,CRTX


In [30]:
def comments_to_df(top_ticks,tick_comments,tick_dates,save,time):
    #restructures dictionaries to useful dataframe format
    #save indicates whether we want to save the dataframe
    #daily or weekly indicate what your csv represents
    d = {"tick":[],"comment":[],"date":[]}
    for tick in top_ticks:
        for i in range(len(tick_comments[tick])):
            d["tick"].append(tick)
            d["comment"].append(tick_comments[tick][i])
            d["date"].append(tick_dates[tick][i])
    df = pd.DataFrame(d)
    
     
    if save==True:
        df.to_csv(f"{time}_ticks_comments.csv", index=False)        
    return df
            
            
            

In [31]:
comments_to_df(weekly_ticks,tick_comments[(0,0)],tick_dates[(0,0)],True,"weekly")

,tick,comment,date
0,AAPL,So AAPL reached ATH and decided to drop 10 buc...,2021-12-02 08:28:02
1,AAPL,AAPL dropped fluff news about their car before...,2021-12-02 07:25:08
2,AAPL,AAPL looking like the russel false breakout. ...,2021-12-02 06:35:51
3,AAPL,Bought AAPL 12/10 170p yesterday when it was A...,2021-12-02 07:28:53
4,AAPL,"So, the megachamp AAPL that’s been propping up...",2021-12-02 07:32:39
...,...,...,...
2136,PE,Filtering using PE leads to some misconception...,2021-11-25 12:04:29
2137,PE,Filtering using PE leads to some misconception...,2021-11-25 12:04:29
2138,PE,Filtering using PE leads to some misconception...,2021-11-25 12:04:29
2139,PE,"I mean, how many of these are in industries wh...",2021-11-25 12:08:25


In [32]:
comments_to_df(daily_ticks,tick_comments_d[(0,0)],tick_dates_d[(0,0)],True,"daily")

,tick,comment,date
0,CRTX,Ask all the people who sold CRTX puts in Octob...,2021-12-02 09:41:17
1,CRTX,https://www.marketwatch.com/tools/screener/sho...,2021-12-02 08:06:11
2,CRTX,The only thing with CRTX is that the shorts en...,2021-12-02 09:12:46
3,CRTX,CRTX to the moon!,2021-12-01 16:08:45
4,CRTX,CRTX 🚀🚀🚀🚀🌚🌚🌚🌚,2021-12-01 17:48:05
...,...,...,...
547,NVDA,META / AMD / NVDA \n\n\n\*Shareholder,2021-12-02 06:42:34
548,NVDA,Weird how AMD and NVDA are going down. I am n...,2021-12-01 13:14:29
549,NVDA,Weird how AMD and NVDA are going down. I am n...,2021-12-01 13:14:29
550,NVDA,I thought the same thing half a year ago. I am...,2021-12-01 19:39:10


In [ ]:
#SIA.lexicon.update(new_words)